# Regression Adjustment
* d分離を考慮した回帰モデルの構築
* 結果変数が連続量の時は線形回帰，離散量の時はロジスティック回帰
* d分離の操作の後に残っている変数を入力変数として回帰モデルを構築する
* データの関係性が線形なことを仮定している

In [1]:
from sklearn.linear_model import LinearRegression

import random
import numpy as np
import pandas as pd


from numpy.random import *
import matplotlib.pyplot as plt
import scipy.stats
from scipy.special import expit


np.random.seed(3655)

## データの用意
* d分離をした後に残っている変数を使い回帰モデルを構築する(今回はそのような擬似データを作る)

In [2]:
# データ数
num_data = 100000

# 年齢
x_1 = randint(15, 76, num_data)  # 15から75歳の一様乱数

# 性別（0を女性、1を男性とします）
x_2 = randint(0, 2, num_data)  # 0か1の乱数

# ノイズの生成
e_z = randn(num_data)

# シグモイド関数に入れる部分
z_base = x_1 + (1-x_2)*10 - 40 + 5*e_z

# シグモイド関数を計算
z_prob = expit(0.1*z_base)

# テレビCMを見たかどうかの変数（0は見ていない、1は見た）
Z = np.array([])

for i in range(num_data):
    Z_i = np.random.choice(2, size=1, p=[1-z_prob[i], z_prob[i]])[0]
    Z = np.append(Z, Z_i)
    
# ノイズの生成
e_y = randn(num_data)

Y = -x_1 + 30*x_2 + 10*Z + 80 + 10*e_y

In [3]:
df = pd.DataFrame({'年齢': x_1,
                   '性別': x_2,
                   'CMを見た': Z,
                   '購入量': Y,
                   })

df.head() 

,年齢,性別,CMを見た,購入量
0,42,1,1.0,80.742174
1,46,1,1.0,78.582308
2,28,0,1.0,69.532577
3,21,0,0.0,73.698435
4,69,1,1.0,44.185729


In [4]:
print(df[df["CMを見た"] == 1.0].mean())
print(df[df["CMを見た"] == 0.0].mean())

年齢       52.512092
性別        0.445553
CMを見た     1.000000
購入量      50.823187
dtype: float64
年齢       31.486615
性別        0.598827
CMを見た     0.000000
購入量      66.512602
dtype: float64


## 因果の効果

In [5]:
# 説明変数
X = df[["年齢", "性別", "CMを見た"]]

# 被説明変数（目的変数）
y = df["購入量"]

# 回帰の実施
reg = LinearRegression().fit(X, y)

# 回帰した結果の係数を出力
print("係数：", reg.coef_)


係数： [-0.99893762 29.97204626  9.90764621]
